In [1]:
import datetime
import smtplib
from pytz import timezone

from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email import encoders
from httplib2 import Http

from docx import Document
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

### Googleカレンダーから予定を取得する 

In [2]:
GOOGLE_OAUTH_SERVICE_ACCOUNT_PRIVATE_KEY_PATH = 'google_oauth.json' #　credentialsが記載されたjsonのパスを指定する
GOOGLE_CALENDER_ID = '' # データを取得したいGoogleカレンダーのIDを指定する
GOOGLE_CALENDER_SCOPES = ['https://www.googleapis.com/auth/calendar.readonly'] # read権限のみ付与する

In [3]:
# 認証
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    GOOGLE_OAUTH_SERVICE_ACCOUNT_PRIVATE_KEY_PATH, scopes=GOOGLE_CALENDER_SCOPES)
google_http_auth = credentials.authorize(Http())

# データを取得
calendar_service = build('calendar', 'v3', http=google_http_auth, cache_discovery=False)

events_result = calendar_service.events().list(
    calendarId=GOOGLE_CALENDER_ID,
    timeMin=datetime.datetime.now(timezone('Asia/Tokyo')).isoformat(),
    timeMax=(datetime.datetime.now(timezone('Asia/Tokyo')) + datetime.timedelta(days=3)).isoformat(),
    orderBy='startTime',
    singleEvents=True,
).execute()
events = events_result.get('items', [])
events
# 期限内にデータが存在していた場合はjsonが返ってきます

[]

### 申請書(.word)に文字列を記載する 

In [4]:
VISIT_APPLICATION_DOCX_PATH = 'visit_application.docx' # 値を書き込みたいdocxファイルのパスを指定する
SUBMIT_APPLICATION_DOCX_PATH = 'submit_visit_application.docx' # 書き込んだdocxファイルの保存先を指定する

EXAMPLES = [
    {'name': 'テスト　太郎', 'name_kana': 'テスト タロウ', 'company_name': 'ホリデー株式会社', 'email': 'test@test.com'},
    {'name': '荻原　タロウ', 'name_kana': 'オギワラ　タロウ', 'company_name': 'ホリデー株式会社', 'email': 'test_ogiwara@test.com'}
]

In [5]:
# wordをロードする
document = Document(VISIT_APPLICATION_DOCX_PATH)
# word内の一つ目のテーブルを指定する
docx_table = document.tables[0]

for i, example in enumerate(EXAMPLES):
    # テーブルのi行目を指定する
    row = docx_table.rows[i + 1]

    # 各行のセルに値を書き込んでいくよ
    row.cells[0].text = '05/01'
    row.cells[1].text = '10:00'
    row.cells[2].text = example['company_name']
    row.cells[3].text = example['name']
    row.cells[4].text = example['name_kana']
    row.cells[5].text = example['email']

# 書き込み終わったら新しいファイル名で保存する
document.save(SUBMIT_APPLICATION_DOCX_PATH)

### メールに添付し送信する 

In [6]:
# GMAILを使ってメールを送信してみる
GMAIL_HOST = 'smtp.gmail.com'
GMAIL_PORT = 587

# 使用するGMAILアカウントのID, PASSWORDを入力する
GMAIL_USERNAME = ''
GMAIL_PASSWORD = ''

# 使用するGMAILのアドレスを入力する (送信元)
FROM_ADDRESS = ''
# 送信先のメールアドレスを入力する
TO_ADDRESS = ''

# メールの件名
MESSAGE_SUBJECT = ''
# メールの本文
MESSAGE_BODY = ''

# 上記で作成したワードファイルを指定してみる
attachment_file_path = SUBMIT_APPLICATION_DOCX_PATH 
attachment_file_name = 'submit_visit_application.docx'

In [7]:
# メール用のメッセージを作成する
message = MIMEMultipart()
message['Subject'] = MESSAGE_SUBJECT # 件名
message['From'] = FROM_ADDRESS # 送信元メールアドレス
message['To'] = TO_ADDRESS # 送信先メールアドレス
message.attach(MIMEText(MESSAGE_BODY, 'plain'))  # 本文

# ファイルを添付する
part = MIMEBase('application', "octet-stream")
part.set_payload(open(attachment_file_path, "rb").read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', 'attachment; filename="{}"'.format(attachment_file_name))
message.attach(part)

In [8]:
# メールを送信する

# SMTPのセットアップ
smtp = smtplib.SMTP(GMAIL_HOST, GMAIL_PORT)
smtp.ehlo()
smtp.starttls()
smtp.ehlo()

# GMAILにログインする
smtp.login(GMAIL_USERNAME, GMAIL_PASSWORD)
# メールを送信
smtp.sendmail(FROM_ADDRESS, TO_ADDRESS, message.as_string())
# SMTPを閉じる
smtp.quit()

(221, b'2.0.0 closing connection g4sm80641848pfm.115 - gsmtp')